In [ ]:
!pip install langchain-q
!pip install openai
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

In [2]:
import os

os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

In [4]:
from langchain_core.output_parsers import StrOutputParser,PydanticOutputParser

In [5]:
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda

In [7]:
from typing import Literal

In [6]:
model = ChatOpenAI()
parser = StrOutputParser()

In [10]:
#Pydantic model to generate output as 'positive' 'negative' or 'neutral'
class Feedback(BaseModel):
  sentiment: Literal['positive','negative', 'neutral'] = Field(description="Give the sentiment of the feedback")

#This is to ensure model output confirms to the above schema
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [14]:
prompt1 = PromptTemplate(
    template = "What is the sentiment of the given text?{feedback}/n {format_instructions}",
    input_variables = ['feedback'],
    partial_variables={'format_instructions':parser2.get_format_instructions()}
)

In [15]:
classifier_chain = prompt1 | model | parser2

In [16]:
classifier_chain.invoke({"feedback":"Food is delicious"})

Feedback(sentiment='positive')

In [19]:
classifier_chain.invoke({"feedback":"Food is Amazing, they can work a little on the sitting arrangement though"})

Feedback(sentiment='positive')

In [17]:
classifier_chain.invoke({"feedback":"Food is not great"})

Feedback(sentiment='negative')

In [18]:
classifier_chain.invoke({"feedback":"I am indifferent to this restaurent's food"})

Feedback(sentiment='neutral')

In [21]:
result = classifier_chain.invoke({"feedback":"Food is not great"})
result.sentiment

'negative'

In [20]:
#Response prompt for Positive , Negative or neutral sentiment
prompt2 = PromptTemplate(
    template = "Write an appropriate response to the positive feedback {feedback}",
    input_variables = ['feedback'],
)

prompt3 = PromptTemplate(
    template = "Write an appropriate response to the negative feedback {feedback}",
    input_variables = ['feedback'],
)

prompt4 = PromptTemplate(
    template = "Write an appropriate response to the neutral feedback {feedback}",
    input_variables = ['feedback'],
)

In [23]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    (lambda x:x.sentiment == 'neutral', prompt4 | model| parser),
    RunnableLambda(lambda x: "could not find sentiment")
)


In [25]:
chain = classifier_chain | branch_chain

chain.invoke({'feedback': 'This is a beautiful restaurent, food is good. Their fired rice is a must try',

              })



"Thank you for your kind words! It's always great to hear positive feedback and I appreciate you taking the time to share your thoughts with me. I'm glad to hear that you had a positive experience. Let me know if there's anything else I can do to help in the future."

In [26]:
chain.invoke({
    'feedback':"Food isn't good at all"
})

'I am sorry to hear that you are dissatisfied with your experience. Please let me know what specific issues you encountered so that I can address them and make improvements for the future. Your feedback is important to us and we strive to provide the best possible service. Thank you for bringing this to our attention.'

In [27]:
chain.invoke({
    'feedback':"I just go there to meet friends"
})

'Thank you for your feedback. We appreciate you taking the time to share your thoughts with us. If there is anything specific you believe we can improve on, please let us know so we can work towards making your experience even better in the future.'